In [2]:
import os

In [3]:
%pwd

'c:\\Users\\rahul\\Downloads\\Facial_review_system\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\rahul\\Downloads\\Facial_review_system'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_weights: str
    params_classes: int
    
@dataclass(frozen=True)
class PrepareWeightsConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from Facial_Review_System.constants import *
from Facial_Review_System.utils.common import read_yaml, create_directories

In [8]:
#this class for base model configuration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = VGGFACE_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config
    
#this class is for weights configuration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = WEIGHTS_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_weights_config(self) -> PrepareWeightsConfig:
        config = self.config.prepare_weights

        create_directories([config.root_dir])
        
        prepare_weights_config = PrepareWeightsConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )


        return prepare_weights_config

In [9]:
import os
import zipfile
import gdown
from Facial_Review_System import logger
from Facial_Review_System.utils.common import get_size

In [10]:
#this class is for model download
class ModelDownload:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/prepare_base_model", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
#this class if for weights download

class WeightsDownload:
    def __init__(self, config: PrepareWeightsConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/prepare_base_model", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    model_config = config.get_prepare_weights_config()
    vgg_model = ModelDownload(config = model_config) #change the variable in 2nd line paramenter name dumboo
    vgg_model.download_file()
    vgg_model.extract_zip_file()
except Exception as e:
    raise e

[2024-04-25 22:59:04,214: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-25 22:59:04,218: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-25 22:59:04,219: INFO: common: created directory at: artifacts]
[2024-04-25 22:59:04,222: INFO: common: created directory at: artifacts/prepare_weights]
[2024-04-25 22:59:04,224: INFO: 3376143486: Downloading data from https://drive.google.com/file/d/12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ/view?usp=sharing into file artifacts/prepare_weights/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ&confirm=t&uuid=1b21ddf6-fbcf-4fac-894d-de102f6de638
To: c:\Users\rahul\Downloads\Facial_review_system\artifacts\prepare_weights\data.zip
100%|██████████| 598M/598M [01:44<00:00, 5.73MB/s] 

[2024-04-25 23:00:57,054: INFO: 3376143486: Downloaded data from https://drive.google.com/file/d/12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ/view?usp=sharing into file artifacts/prepare_weights/data.zip]


BadZipFile: File is not a zip file

In [13]:
try:
    config = ConfigurationManager()
    weight_config = config.get_prepare_weights_config()
    comp_weights = WeightsDownload(config=weight_config)#change the variable in above line paramenter name dumboo
    comp_weights.download_file()
    comp_weights.extract_zip_file()
except Exception as e:
    raise e

[2024-04-25 23:46:47,820: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-25 23:46:47,822: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-25 23:46:47,823: INFO: common: created directory at: artifacts]
[2024-04-25 23:46:47,825: INFO: common: created directory at: artifacts/prepare_weights]
[2024-04-25 23:46:47,825: INFO: 1767681583: Downloading data from https://drive.google.com/file/d/12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ/view?usp=sharing into file artifacts/prepare_weights/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ&confirm=t&uuid=c24cec1e-7e84-4c3f-9d1d-eff8680cf741
To: c:\Users\rahul\Downloads\Facial_review_system\artifacts\prepare_weights\data.zip
100%|██████████| 598M/598M [01:39<00:00, 5.99MB/s] 


[2024-04-25 23:48:30,155: INFO: 1767681583: Downloaded data from https://drive.google.com/file/d/12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ/view?usp=sharing into file artifacts/prepare_weights/data.zip]


BadZipFile: File is not a zip file

In [ ]:
import urllib.request as request
import tensorflow as tf